# Evaluation of SIFT scale transformation uncertatinty
0) Specify datasets

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from sift_uncertainty import *

scenes = ['Alamo', 'Ellis_Island', 'Madrid_Metropolis', 'NYC_Library', 'Piazza_del_Popolo', 'Roman_Forum', 'Tower_of_London', 'Union_Square', 'Vienna_Cathedral', 'Yorkminster']
scales = [3.97745, 0.94967, 14.44161, 6.95844, 6.25074, 24.38904, 15.72399, 6.85125, 15.07437, 13.26471]

1) Load all the keypoints (angles, scales, positions) and GT homographies

In [ ]:
data = {'sift_a1':[], 'sift_a2': [], 'sift_s1': [], 'sift_s2': [], 'sift_u1': [], 'sift_u2': [], 'gt_H1to2': [], 'gt_Hids': []}
for scene, scale in zip(scenes, scales):
    data = collect_all(scene, scale, data)

2. Calculate the SIFT scale transformations

In [ ]:
sift_s12 = np.array(data['sift_s2']) / np.array(data['sift_s1'])

plt.hist(sift_s12, 40, range=(0,6))
plt.xlabel(r'The detector scale transformation $r_i$')
plt.ylabel('Occurrence')
plt.title('Histogram of the detector scale transformation')
plt.yscale('log')
plt.savefig('histogram_of_detector_scale_transformation.png', dpi=100)
plt.show()

3) Aproximate the $H_i$ localy by the affinity matrix $A_i$

In [ ]:
ACs = [[] for i in range(len(data['sift_a1']))]

H2pts_ids = [[] for j in range(len(data['gt_Hids']))]
for j,x in enumerate(data['gt_Hids']):
    H2pts_ids[x].append(j)

for i in tqdm(range(len(data['gt_H1to2']))):
    ids = H2pts_ids[i]
    H = data['gt_H1to2'][i]
    u1 = np.array([data['sift_u1'][j] for j in ids])
    u2 = np.array([data['sift_u2'][j] for j in ids])
    ACs_arr = affines_from_homography(H, u1, u2)
    for j, id in enumerate(ids):
        ACs[id] = ACs_arr[j,:,:]

4) Decompose $A_i$ it to the angles by QT, SVD, and exponential analysis

In [ ]:
gt_a12_svd, gt_a12_logm, gt_s12, cond_num_svd, shear_mag_logm = decompose_affines(ACs)

6) Filter the scale transformations with condition number > 1.2

In [ ]:
filter = np.array(cond_num_svd) < 1.2
sift_s12_filtered = np.array(sift_s12)[filter]
gt_s12_filtered = np.array(gt_s12)[filter]

7) Calculate the translation scale error & filter outliers

In [ ]:
d_s12 = sift_s12_filtered / gt_s12_filtered

plt.hist(d_s12, 40, range=(0.25,1.75))
plt.xlabel(r'The detector scale transformation error $\Delta_r$')
plt.ylabel('Occurrence')
plt.title(r'Histogram of $\Delta_r$ before filtering outliers')
plt.show()

d_s12_filter = (d_s12 > 0.5) & (d_s12 < 2)     # filter points with larger error than 50%
wd_ls12 = np.log(d_s12[d_s12_filter]) / gt_s12_filtered[d_s12_filter]

8) Plot the histograms of scale transformation errors

In [ ]:
plt.hist(d_s12, 40, range=(0.25,1.75))
plt.xlabel(r'$\Delta_{r_i}}$')
plt.ylabel('Occurrence')
plt.title(r'Histogram of the scale transformation error')
plt.savefig('histogram_of_scale_transformation_error.png', dpi=100)
plt.show()

plt.hist(wd_ls12, 40, range=(-0.75,0.75))
plt.xlabel(r'$\rho_i$')
plt.ylabel('Occurrence')
plt.title('Histogram of weighted log-scale transformation error')
plt.savefig('histogram_of_weighted_logscale_transformation_error.png', dpi=100)
plt.show()
